In [2]:
import pandas as pd
import numpy as np
import requests
import json

In [ ]:
# Constants
SENSEBOX_ID = "6793f4e4c326f20007c34dd2"  # The ID of the SenseBox
API_URL_FORMAT_BOX = "https://api.opensensemap.org/boxes/{sensebox_id}?format={response_format}"
API_URL_FORMAT_SENSOR = "https://api.opensensemap.org/boxes/{sensebox_id}/data/{sensor_id}?from-date={fromDate}&to-date={toDate}&download=true&format=json"

# Function to create and return a dictionary {sensor name : sensor id}
def get_sensor_names_ids():
    # Construct the URL for the API request using the SenseBox ID and JSON format
    url = API_URL_FORMAT_BOX.format(sensebox_id=SENSEBOX_ID, response_format="json")

    # Ensure the API request is successful by checking the status code
    status_code = requests.get(url).status_code
    assert status_code, f"Failed fetching data from API. Status code: {status_code}"

    # Get the 'sensors' list from the JSON response
    sensors = requests.get(url).json().get("sensors")

    # Create a dictionary to hold sensor names and their corresponding IDs
    sensor_name_id = {}
    for sensor in sensors:
        # Format the sensor name: remove spaces and replace special characters
        name = sensor.get('title').replace(" ", "").replace(".", "_").replace("-", "_")
        _id = sensor.get('_id')  # Get the sensor ID
        sensor_name_id.update({name: _id})  # Add entry to the dictionary

    # Return the dictionary of sensor names and IDs
    return sensor_name_id

### So kann man die namen und ids als eine Liste haben

In [73]:
get_sensor_names_ids()

{'Temperatur': '6793f4e4c326f20007c34dd3',
 'rel_Luftfeuchte': '6793f4e4c326f20007c34dd4',
 'Beleuchtungsstärke': '6793f4e4c326f20007c34dd5',
 'UV_Intensität': '6793f4e4c326f20007c34dd6',
 'PM1': '6793f4e4c326f20007c34dd7',
 'PM2_5': '6793f4e4c326f20007c34dd8',
 'PM4': '6793f4e4c326f20007c34dd9',
 'PM10': '6793f4e4c326f20007c34dda'}

### Dict Ausgabe von der Funktion

In [5]:
get_sensor_names_ids()

{'Temperatur': '6793f4e4c326f20007c34dd3',
 'rel_Luftfeuchte': '6793f4e4c326f20007c34dd4',
 'Beleuchtungsstärke': '6793f4e4c326f20007c34dd5',
 'UV_Intensität': '6793f4e4c326f20007c34dd6',
 'PM1': '6793f4e4c326f20007c34dd7',
 'PM2_5': '6793f4e4c326f20007c34dd8',
 'PM4': '6793f4e4c326f20007c34dd9',
 'PM10': '6793f4e4c326f20007c34dda'}

### So kann man auf die einzelne Namen zugreifen

In [6]:
sensor_id = get_sensor_names_ids()["Temperatur"]
sensor_id

'6793f4e4c326f20007c34dd3'

In [70]:
from datetime import datetime, timezone, timedelta
def get_data(sensor_id:str,fromDate,toDate):

    #get URL with sensor_id and dates
    url = API_URL_FORMAT_SENSOR.format(sensebox_id=SENSEBOX_ID, sensor_id=sensor_id, fromDate=fromDate, toDate=toDate)
    
    #Make sure we get correct response
    status_code = requests.get(url).status_code
    assert status_code == 200, f"Failed fetching data from api {status_code}"
    
    #Get the json file of the sensor
    data = requests.get(url).json()
    data = [{'createdAt': item['createdAt'], 'value': item['value']} for item in data]
    return data


In [71]:
now = datetime.now(timezone.utc)
iso_now = now.isoformat().replace('+00:00','Z')

two_weeeks_ago = datetime.now(timezone.utc) - timedelta(minutes=10) 
iso_two_weeks_ago = two_weeeks_ago.isoformat().replace('+00:00','Z')

iso_now, iso_two_weeks_ago

data = get_data(sensor_id="6793f4e4c326f20007c34dd3",fromDate=iso_two_weeks_ago,toDate=iso_now)
data

[{'createdAt': '2025-06-27T20:59:59.023Z', 'value': '25.53'},
 {'createdAt': '2025-06-27T20:58:54.758Z', 'value': '25.57'},
 {'createdAt': '2025-06-27T20:57:54.621Z', 'value': '25.56'},
 {'createdAt': '2025-06-27T20:56:56.866Z', 'value': '25.59'},
 {'createdAt': '2025-06-27T20:55:54.347Z', 'value': '25.61'},
 {'createdAt': '2025-06-27T20:54:54.006Z', 'value': '25.60'},
 {'createdAt': '2025-06-27T20:53:53.754Z', 'value': '25.60'},
 {'createdAt': '2025-06-27T20:52:53.480Z', 'value': '25.57'},
 {'createdAt': '2025-06-27T20:51:53.225Z', 'value': '25.57'},
 {'createdAt': '2025-06-27T20:50:52.961Z', 'value': '25.66'}]

,createdAt,value
0,2025-06-27T18:25:22.088Z,28.67
1,2025-06-27T18:24:21.477Z,28.77
2,2025-06-27T18:23:21.331Z,28.92
3,2025-06-27T18:22:21.145Z,29.02
4,2025-06-27T18:21:30.991Z,29.18
5,2025-06-27T18:20:20.814Z,29.14
6,2025-06-27T18:19:20.767Z,29.32
7,2025-06-27T18:18:20.543Z,29.37
8,2025-06-27T18:17:20.542Z,29.49
9,2025-06-27T18:16:20.260Z,29.48


### create_dataframe function mit inputs sensor_id und from- und toDate die einen default Wert haben

In [ ]:
def create_dataframe(sensor_id, fromDate="2025-04-10T00:00:00Z", toDate="2025-04-24T00:00:00Z"):
    url = API_URL_FORMAT_SENSOR.format(sensebox_id=SENSEBOX_ID, sensor_id=sensor_id, fromDate=fromDate, toDate=toDate)
    
    status_code = requests.get(url).status_code
    assert status_code == 200, f"Failed fetching data from api {status_code}"
    
    data = requests.get(url).json()
    return pd.DataFrame(data)


In [14]:
df = create_dataframe(sensor_id=get_sensor_names_ids()["rel_Luftfeuchte"])
df


,location,createdAt,value
0,"[-48.480915, -1.299151]",2025-04-23T23:59:26.693Z,96.27
1,"[-48.480915, -1.299151]",2025-04-23T23:58:26.555Z,96.96
2,"[-48.480915, -1.299151]",2025-04-23T23:57:26.346Z,96.56
3,"[-48.480915, -1.299151]",2025-04-23T23:56:25.902Z,96.56
4,"[-48.480915, -1.299151]",2025-04-23T23:55:25.714Z,96.47
...,...,...,...
9995,"[-48.480915, -1.299151]",2025-04-16T22:18:29.056Z,97.71
9996,"[-48.480915, -1.299151]",2025-04-16T22:17:29.008Z,97.75
9997,"[-48.480915, -1.299151]",2025-04-16T22:16:35.293Z,97.41
9998,"[-48.480915, -1.299151]",2025-04-16T22:15:28.203Z,97.55
